<a href="https://colab.research.google.com/github/sravan1320/RAG/blob/main/LLAMA2_RAG_FIASSDB_Chat_With_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install transformers langchain torch constants chromadb accelerate sentence-transformers pypdf pdfminer.six llama-cpp-python --upgrade huggingface_hub unstructured==0.7.12 InstructorEmbedding faiss-gpu
# !pip install faiss-gpu

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [5]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [7]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/CodeLlama-13B-Python-GGUF"
model_basename = "codellama-13b-python.Q5_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py:337: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


codellama-13b-python.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [8]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--CodeLlama-13B-Python-GGUF/snapshots/c9b66de0e0716d0515f4a86362fd64646a035df6/codellama-13b-python.Q5_K_M.gguf'

In [9]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
model_path=model_path,
max_tokens=256,
n_gpu_layers=512,
n_batch=30,
callback_manager=callback_manager,
n_ctx=1024,
verbose=True,
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [11]:
loader = UnstructuredFileLoader(
    "docs/2018-Annual-Report.pdf", strategy="fast", mode="elements"
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py:337: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
db = FAISS.from_documents(docs, hf_embedding)

# save embeddings in local directory
db.save_local("db/faiss_AiArticle")

# load from local
db = FAISS.load_local("db/faiss_AiArticle/", embeddings=hf_embedding)

In [13]:
query = "what is quaterly results (unaudited)?"
search = db.similarity_search(query, k=5)

In [14]:
search

[Document(page_content='Note 11 — QUARTERLY RESULTS (UNAUDITED)', metadata={'source': 'docs/2018-Annual-Report.pdf', 'filename': '2018-Annual-Report.pdf', 'file_directory': 'docs', 'filetype': 'application/pdf', 'page_number': 77, 'category': 'Title'}),
 Document(page_content='The following tables contain selected unaudited statement of operations information for each quarter of 2017 and 2018. The following information reflects all normal recurring adjustments necessary for a fair presentation of the information for the periods presented. The operating results for any quarter are not necessarily indicative of results for any future period. Our business is affected by seasonality, which historically has resulted in higher sales volume during our fourth quarter. Unaudited quarterly results are as follows (in millions, except per share data):', metadata={'source': 'docs/2018-Annual-Report.pdf', 'filename': '2018-Annual-Report.pdf', 'file_directory': 'docs', 'filetype': 'application/pdf', 

In [15]:
template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context only
do not use general knowledge to answer the query'''

prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

In [ ]:
llm_chain.run(final_prompt)

In [ ]:
##references
https://medium.com/@mayuresh.gawai/implementation-of-llama-v2-in-python-using-langchain-%EF%B8%8F-ebebe82e881b